In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

from scipy import stats
from sklearn import datasets
from sklearn.semi_supervised import label_propagation

from sklearn.metrics import confusion_matrix, classification_report 


In [ ]:
digits = datasets.load_digits()
rng = np.random.RandomState(0) # sets the seed()
indices = np.arange(len(digits.data))  # provides equally spaced intervals between the length of the data digits
rng.shuffle(indices)

print(rng)
print(indices)
print(rng.shuffle(indices))


In [ ]:
X = digits.data[indices[:200]]
y = digits.target[indices[:200]]
images = digits.images[indices[:200]]

n_total_samples = len(y)
n_labeled_points = 30

indices = np.arange(n_total_samples)

unlabeled_set = indices[n_labeled_points:]

# shuffle everything around
y_train = np.copy(y)
y_train[unlabeled_set] = -1

In [ ]:
label_propogation_model = label_propagation.LabelSpreading(gamma=0.25, max_iter=5)
label_propogation_model.fit(X,y_train)

predicted_models = label_propogation_model.transduction_[unlabeled_set] 
true_labels = y[unlabeled_set]

confusion_matrix_data = confusion_matrix(true_labels, predicted_models, labels = label_propogation_model.classes_) 

print("Label Spreading model: %d labeled & %d unlabeled points (%d total)" % (n_labeled_points, n_total_samples - 
                                                                              n_labeled_points, n_total_samples)) 
print(classification_report(true_labels, predicted_models))

                                                            


In [ ]:
print "Confusion Matrix:"
print confusion_matrix_data

In [ ]:
prediction_entropies = stats.distributions.entropy(label_propogation_model.label_distributions_.T) 
uncertainity = np.argsort(prediction_entropies)[-10:]

In [51]:
figure = plt.figure(figsize=(7,5))
for index, image_index in enumerate(uncertainity):
    image = images[image_index]
    sub = figure.add_subplot(2,5,index+1) 
    sub.imshow(image,cmap=plt.cm.gray_r)
    plt.xticks([])
    plt.yticks([])
    sub.set_title('predict: %i\ntrue: %i' % (label_propogation_model.transduction_[image_index], y[image_index]))
figure.suptitle('Semi supervised learning')
plt.show() 